In [1]:
import sys
import glob

import numpy as np

sys.path.append('/Users/ikedamorito/Desktop/Gunosy/assignment')
from utils import Tokenizer, NaiveBayes

In [2]:
cat_pathes = glob.glob('data/*/articles.txt')

In [3]:
cat_pathes

['data/IT・科学/articles.txt',
 'data/おもしろ/articles.txt',
 'data/エンタメ/articles.txt',
 'data/グルメ/articles.txt',
 'data/コラム/articles.txt',
 'data/スポーツ/articles.txt',
 'data/国内/articles.txt',
 'data/海外/articles.txt']

In [4]:
category = [path.split('/')[1] for path in cat_pathes]
cat_nums = {}
for i, cat in enumerate(category):
    cat_nums[cat] = i

In [5]:
cat_nums

{'IT・科学': 0,
 'おもしろ': 1,
 'エンタメ': 2,
 'グルメ': 3,
 'コラム': 4,
 'スポーツ': 5,
 '国内': 6,
 '海外': 7}

In [6]:
docs = []
labels = []

In [7]:
t = Tokenizer(dic='/usr/local/lib/mecab/dic/mecab-ipadic-neologd/', stopword=True)

for file_path in cat_pathes:
    cat = file_path.split('/')[1]
    label = cat_nums[cat]
    with open(file_path, 'r') as f:
        texts = f.readlines()
        for text in texts:
            words = t.tokenize(text)
            docs.append(words)
            labels.append(label)

docs = np.array(docs).reshape((-1, 1))
labels = np.array(labels).reshape((-1, 1))

data = np.hstack((docs, labels))
np.random.shuffle(data)

In [8]:
import pickle
with open('../pkl_objects/processed_data.pkl', 'wb') as f:
    pickle.dump(data, f)

In [9]:
with open('../pkl_objects/processed_data.pkl', 'rb') as f:
    data = pickle.load(f)

X = data[:, 0]
y = data[:, 1]

from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

/Users/ikedamorito/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
# ナイーブベイズ分類
nb = NaiveBayes(alpha=1)
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)

In [11]:
y_test = list(y_test)
y_pred = list(y_pred)

In [15]:
# 精度で評価
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print('accuracy: {0}'.format(accuracy))

accuracy: 0.8268281740690893


In [16]:
# 学習した分類器をシリアライズ
import pickle
pickle.dump(nb, open('../pkl_objects/naivebayes.pkl', 'wb'), protocol=4)

## グリッドサーチとCVによるチューニング

In [ ]:
# グリッドサーチ用のパラメータ

# dics = ['/usr/local/lib/mecab/dic/mecab-ipadic-neologd/', '/usr/local/lib/mecab/dic/ipadic/']

In [ ]:
# 最も良かったモデルをpickleで保存
# それをwebアプリに搭載
# できればweb上でモデルのリアルタイム更新を可能にする